In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [2]:
np.random.seed(2024)
dates_ventes = pd.date_range('2023-01-01', '2024-12-31', freq='D')
n_jours = len(dates_ventes)
 
df_ventes = pd.DataFrame({
    'Date': dates_ventes,
    'Ventes': np.random.poisson(1000, n_jours) + np.sin(np.arange(n_jours) * 2 * np.pi / 365) * 200,
    'Region': np.random.choice(['Nord', 'Sud', 'Est', 'Ouest'], n_jours),
    'Produit': np.random.choice(['Produit A', 'Produit B', 'Produit C'], n_jours),
    'Prix_Unitaire': np.random.uniform(50, 200, n_jours)
})
 
df_ventes['CA'] = df_ventes['Ventes'] * df_ventes['Prix_Unitaire']
df_ventes['Mois'] = df_ventes['Date'].dt.to_period('M').astype(str)
 
print(df_ventes.head())

        Date       Ventes Region    Produit  Prix_Unitaire             CA  \
0 2023-01-01  1008.000000    Sud  Produit A     183.985392  185457.274774   
1 2023-01-02   971.442671   Nord  Produit C      67.529556   65601.092156   
2 2023-01-03   976.884322   Nord  Produit B      98.256816   95985.543007   
3 2023-01-04  1032.323933    Sud  Produit C     177.009342  182730.980610   
4 2023-01-05  1011.760485    Sud  Produit A      64.331385   65087.953473   

      Mois  
0  2023-01  
1  2023-01  
2  2023-01  
3  2023-01  
4  2023-01  


In [3]:
ventes_mensuelles = df_ventes.groupby(["Mois", "Region"])["Ventes"].sum().reset_index()

fig = px.line(ventes_mensuelles, x="Mois", y="Ventes", color="Region", title="Evolution mensuelles des ventes par région", markers=True)
fig.update_layout(hovermode="x unified", xaxis_tickangle=-45)

In [4]:
fig = px.scatter_3d(df_ventes, x="Prix_Unitaire", y="Ventes", z="CA", color="Produit", size="CA", title="Graph 3D : Prix Ventes CA")

fig.update_layout(height=500)

In [5]:
ca_region = df_ventes.groupby(["Region", "Produit"])["CA"].sum().reset_index()

fig = px.bar(ca_region, x="Region", y="CA", color="Produit", title="Répartition du CA par région et produit", barmode="stack", text_auto=".2s")
fig.update_layout(height=500)

In [23]:
ca_mensuel = df_ventes.groupby("Mois").agg({"CA": "sum", "Ventes": "sum"}).reset_index()
top_produits = df_ventes.groupby("Produit")["CA"].sum()
ventes_region = df_ventes.groupby("Region")["Ventes"].sum()

fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=("Evolution CA", "Top produits", "Distribution régionale", "Tendance prix"),
    specs=[
        [{"type": "scatter"}, {"type": "bar"}],
        [{"type": "pie"}, {"type": "scatter"}]
    ]
)

fig.add_trace(go.Scatter(x=ca_mensuel["Mois"], y=ca_mensuel["CA"], name="CA"), row=1, col=1)
fig.add_trace(go.Bar(x=top_produits.values, y=top_produits.index, orientation="h", name="CA produits"), row=1, col=2)
fig.add_trace(go.Pie(labels=ventes_region.index, values=ventes_region.values), row=2, col=1)
fig.add_trace(go.Scatter(x=ca_mensuel["Mois"], y=ca_mensuel["Ventes"], name="Ventes"), row=2, col=2)